In [1]:
import numpy as np
import pandas as pd
import pickle
import geopandas as gpd
from shapely.geometry import Point

import netCDF4
 
import datetime

import psycopg2

import os


# Connect to DB and dowload all the adm_0 GID

In [3]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19',
    user='covid19',
    password='covid19'
)

In [4]:
def get_administrative_division(level=0, countrycode=None):
    sql = f"SELECT * FROM administrative_division WHERE adm_level={level} "
    #if level>0 and countrycode: 
    if countrycode:
        sql = sql +  f"AND countrycode='{countrycode}'"

    admin_data = gpd.GeoDataFrame.from_postgis(sql, conn, geom_col='geometry')
    return admin_data

In [4]:
adm_0 = get_administrative_division(level=0)

In [5]:
adm_0

,country,countrycode,countrycode_alpha2,adm_level,adm_area_1,adm_area_1_code,adm_area_2,adm_area_2_code,adm_area_3,adm_area_3_code,gid,latitude,longitude,properties,geometry
0,Anguilla,AIA,AI,0,None,None,None,None,None,None,AIA,18.223382,-63.058133,"{'GID_0': 'AIA', 'NAME_0': 'Anguilla'}","POLYGON ((-63.06847 18.23680, -62.96514 18.276..."
1,Palestina,PSE,PS,0,None,None,None,None,None,None,PSE,31.916065,35.209176,"{'GID_0': 'PSE', 'NAME_0': 'Palestina'}","MULTIPOLYGON (((34.96923 31.60279, 35.21759 31..."
2,Cyprus,CYP,CY,0,None,None,None,None,None,None,CYP,34.926722,33.003037,"{'GID_0': 'CYP', 'NAME_0': 'Cyprus'}","MULTIPOLYGON (((33.90652 35.06912, 34.02125 35..."
3,Micronesia,FSM,FM,0,None,None,None,None,None,None,FSM,7.081244,154.649175,"{'GID_0': 'FSM', 'NAME_0': 'Micronesia'}","MULTIPOLYGON (((162.96695 5.28111, 162.89722 5..."
4,French Guiana,GUF,GF,0,None,None,None,None,None,None,GUF,3.930452,-53.230951,"{'GID_0': 'GUF', 'NAME_0': 'French Guiana'}","POLYGON ((-51.63115 4.23542, -51.65777 4.05079..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Yemen,YEM,YE,0,None,None,None,None,None,None,YEM,15.904837,47.582759,"{'GID_0': 'YEM', 'NAME_0': 'Yemen'}","MULTIPOLYGON (((53.73915 12.63625, 53.84042 12..."
252,Zambia,ZMB,ZM,0,None,None,None,None,None,None,ZMB,-13.462276,27.792636,"{'GID_0': 'ZMB', 'NAME_0': 'Zambia'}","POLYGON ((25.88753 -17.98407, 25.84223 -17.894..."
253,South Africa,ZAF,ZA,0,None,None,None,None,None,None,ZAF,-28.996293,25.086904,"{'GID_0': 'ZAF', 'NAME_0': 'South Africa'}","POLYGON ((29.21586 -31.93042, 28.92347 -32.243..."
254,Zimbabwe,ZWE,ZW,0,None,None,None,None,None,None,ZWE,-19.000563,29.868838,"{'GID_0': 'ZWE', 'NAME_0': 'Zimbabwe'}","POLYGON ((31.05947 -22.33982, 30.83536 -22.291..."


In [6]:
# dump dict to pickle file
with open('input/adm_0.pkl', 'wb') as handle:
    pickle.dump(adm_0, handle)

# Read the level 0 dicts and put them in a list

In [8]:
! rm out/level_0/.DS_Store

rm: out/level_0/.DS_Store: No such file or directory


In [9]:
dict_list = []

keys = []

for filename in os.listdir(os.getcwd()+"/out/level_0/"):
    with open(os.getcwd()+'/out/level_0/'+filename, 'rb') as f:
        temp = pickle.load(f)
        dict_list.append(temp)

        for k in temp.keys():
            keys.append(k)

In [10]:
len(dict_list)

40

In [11]:
found_countries = pd.unique(keys)
len(found_countries)

234

# Merge the dicts

In [12]:
adm_0_to_grid = {}

for d in dict_list:
    # each value of each dict is a list of list [idx, point]
    for key, value_list in d.items():
        for item in value_list:
            adm_0_to_grid.setdefault(key, []).append(item)



In [13]:
# there are 256 countries in gadm
len(adm_0_to_grid.keys())

234

In [15]:
for key in adm_0_to_grid:
    print(key, len(adm_0_to_grid[key]))

TCD 8133
SDN 11999
ERI 769
YEM 2897
IND 21139
MMR 4393
THA 3272
LAO 1498
VNM 2117
PHL 1851
MEX 13169
GTM 693
HND 709
DMA 4
CPV 24
SEN 1244
MRT 6830
MLI 8068
NER 7645
GLP 9
BLZ 139
SAU 12919
OMN 2039
MSR 1
ATG 3
DOM 311
JAM 70
PRI 57
HTI 172
SXM 1
CHN 72197
AIA 1
VIR 2
VGB 1
DZA 16155
USA 84672
CYM 1
LBY 11203
CHL 5918
HMD 3
NZL 2209
ARG 20904
FLK 121
ATF 72
CAN 128032
FRA 4893
CHE 367
AUT 763
DEU 3488
HUN 846
ROU 2095
MDA 304
UKR 5613
RUS 221450
KAZ 25059
MNG 14013
SVK 452
CZE 749
POL 3111
JEY 1
GGY 1
BEL 292
LUX 23
GBR 2546
NLD 371
BLR 2128
IRL 712
GRL 49614
SJM 1895
ECU 1576
PER 8097
COL 7030
BRA 53788
GAB 1636
COG 2105
COD 14424
BDI 166
TZA 5827
KEN 3617
IDN 11600
PNG 2845
RWA 156
SOM 3915
UGA 1493
STP 8
VEN 5668
MYS 2017
GNQ 167
SGP 5
GUY 1291
BHS 85
ESH 1804
EGY 6743
QAT 80
ARE 475
PAK 6196
BGD 934
TWN 245
IRN 11835
BHR 5
JPN 2864
NPL 1033
BTN 268
ESP 4070
MAR 2992
KWT 121
IRQ 3256
CMR 2884
KIR 3
SUR 909
GUF 512
CAF 3842
SSD 3917
ETH 7039
BRN 34
NGA 5665
LBR 590
CIV 2004
GHA 1482


In [16]:
# we are missing 22 countries
adm_0[~adm_0.countrycode.isin(list(adm_0_to_grid.keys()))]

,country,countrycode,countrycode_alpha2,adm_level,adm_area_1,adm_area_1_code,adm_area_2,adm_area_2_code,adm_area_3,adm_area_3_code,gid,latitude,longitude,properties,geometry
39,Saint-Barthélemy,BLM,BL,0,None,None,None,None,None,None,BLM,17.906343,-62.833741,"{'GID_0': 'BLM', 'NAME_0': 'Saint-Barthélemy'}","MULTIPOLYGON (((-62.90458 17.96153, -62.90458 ..."
40,Nauru,NRU,NR,0,None,None,None,None,None,None,NRU,-0.528470,166.934484,"{'GID_0': 'NRU', 'NAME_0': 'Nauru'}","POLYGON ((166.92877 -0.55353, 166.92828 -0.553..."
65,Cook Islands,COK,CK,0,None,None,None,None,None,None,COK,-19.182795,-158.929475,"{'GID_0': 'COK', 'NAME_0': 'Cook Islands'}","MULTIPOLYGON (((-157.70778 -19.83837, -157.678..."
91,Gibraltar,GIB,GI,0,None,None,None,None,None,None,GIB,36.136503,-5.349238,"{'GID_0': 'GIB', 'NAME_0': 'Gibraltar'}","MULTIPOLYGON (((-5.33903 36.15467, -5.33903 36..."
111,British Indian Ocean Territory,IOT,IO,0,None,None,None,None,None,None,IOT,-6.682924,72.120700,"{'GID_0': 'IOT', 'NAME_0': 'British Indian Oce...","MULTIPOLYGON (((72.43242 -7.38327, 72.42736 -7..."
129,Saint Kitts and Nevis,KNA,KN,0,None,None,None,None,None,None,KNA,17.271858,-62.701341,"{'GID_0': 'KNA', 'NAME_0': 'Saint Kitts and Ne...","MULTIPOLYGON (((-62.57724 17.10264, -62.63292 ..."
132,Liechtenstein,LIE,LI,0,None,None,None,None,None,None,LIE,47.142463,9.553805,"{'GID_0': 'LIE', 'NAME_0': 'Liechtenstein'}","POLYGON ((9.47780 47.05272, 9.52333 47.09419, ..."
147,Macao,MAC,MO,0,None,None,None,None,None,None,MAC,22.162351,113.561331,"{'GID_0': 'MAC', 'NAME_0': 'Macao'}","MULTIPOLYGON (((113.57655 22.13850, 113.57689 ..."
148,Saint-Martin,MAF,MF,0,None,None,None,None,None,None,MAF,18.079513,-63.060262,"{'GID_0': 'MAF', 'NAME_0': 'Saint-Martin'}","POLYGON ((-63.13943 18.05236, -63.01847 18.122..."
150,Monaco,MCO,MC,0,None,None,None,None,None,None,MCO,43.735679,7.423269,"{'GID_0': 'MCO', 'NAME_0': 'Monaco'}","MULTIPOLYGON (((7.41338 43.73459, 7.43630 43.7..."


## dump it to pikle

In [20]:
# dump dict to pickle file
with open('input/dicts/adm_0_to_grid.pkl', 'wb') as handle:
    pickle.dump(adm_0_to_grid, handle)

## load it up

In [19]:
with open('input/dicts/adm_0_to_grid.pkl', 'rb') as handle:
    adm_0_to_grid = pickle.load(handle)

In [20]:
len(adm_0_to_grid.keys())

234

# LEVEL 1

In [5]:
adm_1 = get_administrative_division(level=1)
adm_1.head()

,country,countrycode,countrycode_alpha2,adm_level,adm_area_1,adm_area_1_code,adm_area_2,adm_area_2_code,adm_area_3,adm_area_3_code,gid,latitude,longitude,properties,geometry
0,Afghanistan,AFG,AF,1,Badakhshan,AFG.1_1,None,None,None,None,AFG.1_1,37.032895,71.458977,"{'CC_1': None, 'GID_0': 'AFG', 'GID_1': 'AFG.1...","POLYGON ((71.14804 36.00124, 71.10155 35.95555..."
1,Afghanistan,AFG,AF,1,Badghis,AFG.2_1,None,None,None,None,AFG.2_1,35.139751,63.752696,"{'CC_1': None, 'GID_0': 'AFG', 'GID_1': 'AFG.2...","POLYGON ((63.09734 34.64551, 63.06237 34.69018..."
2,Afghanistan,AFG,AF,1,Baghlan,AFG.3_1,None,None,None,None,AFG.3_1,35.705362,68.713877,"{'CC_1': None, 'GID_0': 'AFG', 'GID_1': 'AFG.3...","POLYGON ((67.35538 34.88549, 67.28322 35.07100..."
3,Afghanistan,AFG,AF,1,Balkh,AFG.4_1,None,None,None,None,AFG.4_1,36.648028,67.134957,"{'CC_1': None, 'GID_0': 'AFG', 'GID_1': 'AFG.4...","POLYGON ((66.42347 35.64057, 66.51625 35.67334..."
4,Afghanistan,AFG,AF,1,Bamyan,AFG.5_1,None,None,None,None,AFG.5_1,34.721419,67.181307,"{'CC_1': None, 'GID_0': 'AFG', 'GID_1': 'AFG.5...","POLYGON ((66.65279 34.00322, 66.67029 34.06174..."


In [6]:
# dump dict to pickle file
with open('input/adm_1.pkl', 'wb') as handle:
    pickle.dump(adm_1, handle)

## Merge the level 1 dicts

In [7]:
! rm out/level_1/.DS_Store

rm: out/level_1/.DS_Store: No such file or directory


In [8]:
adm_1_to_grid = {}


for filename in os.listdir(os.getcwd()+"/out/level_1/"):
    with open(os.getcwd()+'/out/level_1/'+filename, 'rb') as f:
        temp = pickle.load(f)
        adm_1_to_grid.update(temp)


In [9]:
len(adm_1_to_grid.keys())

234

In [10]:
for key in adm_1_to_grid:
    print(key, len(adm_1_to_grid[key]))

BHR 2
RUS 83
SSD 10
MYT 1
FJI 4
ZWE 10
MAR 15
SHN 2
FRO 6
HRV 21
MLT 3
CYM 0
MWI 27
ITA 20
UKR 27
BHS 21
PYF 4
USA 51
GNB 8
AND 4
XCA 0
VUT 6
MMR 15
MOZ 11
SDN 18
DJI 5
TON 1
MTQ 4
SRB 25
NPL 5
GTM 22
GUM 2
ECU 24
BES 1
UGA 58
NGA 37
XNC 5
FSM 2
COG 11
DEU 16
SGP 4
AZE 10
SEN 14
MYS 14
VNM 63
TUR 81
GBR 4
GNQ 6
LKA 25
GUY 10
ETH 11
BRN 4
SPM 1
HKG 3
CZE 14
JEY 1
COD 26
SGS 0
CAN 13
TLS 13
MUS 8
JPN 47
ABW 0
TWN 7
POL 16
PLW 0
PSE 2
GRL 5
IDN 33
NCL 3
SVK 8
VEN 25
SLE 4
SAU 13
MKD 73
MEX 32
SYR 14
CRI 7
FLK 0
CPV 13
AIA 0
EGY 27
BLZ 6
GHA 10
PHL 81
BVT 0
GGY 1
QAT 6
CHE 24
CIV 14
ASM 1
SUR 10
SWZ 4
ESH 4
ISR 7
KHM 25
PRT 20
NZL 18
KEN 47
BTN 20
MSR 1
SLB 9
ARM 11
SYC 0
NLD 14
WLF 1
ERI 6
IRQ 18
BWA 12
LTU 10
SVN 12
HMD 0
XKO 7
SOM 17
SLV 14
SMR 1
MRT 13
VGB 1
DZA 48
AGO 18
STP 2
LBN 8
LBY 21
LAO 18
NAM 13
EST 16
HND 17
DOM 32
HUN 20
CHN 31
TJK 5
AFG 34
VIR 2
GIN 8
REU 4
PRI 47
BMU 1
GRD 2
PRK 14
NOR 19
GEO 12
SWE 21
THA 77
CHL 16
TKM 6
HTI 10
TGO 5
MDA 37
DNK 5
ISL 8
BLR 6
LCA 3
GRC 8
P

In [11]:
# dump dict to pickle file
with open('input/dicts/adm_1_to_grid.pkl', 'wb') as handle:
    pickle.dump(adm_1_to_grid, handle)

# LEVEL 2

In [12]:
adm_2 = get_administrative_division(level=2)
adm_2.head()

,country,countrycode,countrycode_alpha2,adm_level,adm_area_1,adm_area_1_code,adm_area_2,adm_area_2_code,adm_area_3,adm_area_3_code,gid,latitude,longitude,properties,geometry
0,Japan,JPN,JP,2,Osaka,JPN.33_1,Matsubara,JPN.33.21_1,None,None,JPN.33.21_1,34.580434,135.554116,"{'CC_2': None, 'GID_0': 'JPN', 'GID_1': 'JPN.3...","POLYGON ((135.56920 34.55938, 135.56711 34.559..."
1,Japan,JPN,JP,2,Osaka,JPN.33_1,Minoo,JPN.33.22_1,None,None,JPN.33.22_1,34.852099,135.489741,"{'CC_2': None, 'GID_0': 'JPN', 'GID_1': 'JPN.3...","POLYGON ((135.52818 34.82368, 135.45795 34.804..."
2,Japan,JPN,JP,2,Osaka,JPN.33_1,Misaki,JPN.33.23_1,None,None,JPN.33.23_1,34.302148,135.154777,"{'CC_2': None, 'GID_0': 'JPN', 'GID_1': 'JPN.3...","POLYGON ((135.21011 34.30307, 135.18756 34.276..."
3,Japan,JPN,JP,2,Osaka,JPN.33_1,Moriguchi,JPN.33.24_1,None,None,JPN.33.24_1,34.739661,135.582107,"{'CC_2': None, 'GID_0': 'JPN', 'GID_1': 'JPN.3...","POLYGON ((135.58665 34.71895, 135.58463 34.716..."
4,Japan,JPN,JP,2,Osaka,JPN.33_1,Neyagawa,JPN.33.25_1,None,None,JPN.33.25_1,34.764424,135.631182,"{'CC_2': None, 'GID_0': 'JPN', 'GID_1': 'JPN.3...","POLYGON ((135.59201 34.76534, 135.59459 34.766..."


In [13]:
# dump to pickle file
with open('input/adm_2.pkl', 'wb') as handle:
    pickle.dump(adm_2, handle)

## Merge the level 2 dicts

In [2]:
adm_2_to_grid = {}


for filename in os.listdir(os.getcwd()+"/out/level_2/"):
    with open(os.getcwd()+'/out/level_2/'+filename, 'rb') as f:
        temp = pickle.load(f)
        adm_2_to_grid.update(temp)


In [3]:
print(len(adm_2_to_grid.keys()))
print(len(adm_2_to_grid["ITA"].keys()))
print(len(adm_2_to_grid["ITA"]["ITA.20_1"].keys()))


234
20
7


In [6]:
for key in adm_2_to_grid:
    for region in adm_2_to_grid[key]:
        print(key, region, len(adm_2_to_grid[key]), len(adm_2_to_grid[key][region]))

LVA LVA.2_1 5 6
LVA LVA.1_1 5 5
LVA LVA.5_1 5 5
LVA LVA.3_1 5 4
LVA LVA.4_1 5 6
BOL BOL.6_1 9 3
BOL BOL.4_1 9 18
BOL BOL.3_1 9 7
BOL BOL.7_1 9 15
BOL BOL.9_1 9 6
BOL BOL.1_1 9 10
BOL BOL.2_1 9 13
BOL BOL.8_1 9 12
BOL BOL.5_1 9 10
JOR JOR.8_1 12 5
JOR JOR.3_1 12 3
JOR JOR.11_1 12 3
JOR JOR.7_1 12 6
JOR JOR.2_1 12 7
JOR JOR.9_1 12 2
JOR JOR.12_1 12 3
JOR JOR.4_1 12 5
JOR JOR.10_1 12 5
JOR JOR.1_1 12 1
JOR JOR.6_1 12 1
JOR JOR.5_1 12 6
ARE ARE.1_1 6 3
ARE ARE.3_1 6 8
ARE ARE.6_1 6 5
ARE ARE.5_1 6 8
ARE ARE.4_1 6 2
ARE ARE.7_1 6 1
IRL IRL.4_1 26 0
IRL IRL.8_1 26 0
IRL IRL.23_1 26 0
IRL IRL.22_1 26 0
IRL IRL.10_1 26 0
IRL IRL.25_1 26 0
IRL IRL.13_1 26 0
IRL IRL.1_1 26 0
IRL IRL.3_1 26 0
IRL IRL.26_1 26 0
IRL IRL.11_1 26 0
IRL IRL.19_1 26 0
IRL IRL.9_1 26 0
IRL IRL.7_1 26 0
IRL IRL.20_1 26 0
IRL IRL.6_1 26 0
IRL IRL.24_1 26 0
IRL IRL.17_1 26 0
IRL IRL.16_1 26 0
IRL IRL.14_1 26 0
IRL IRL.15_1 26 0
IRL IRL.12_1 26 0
IRL IRL.2_1 26 0
IRL IRL.21_1 26 0
IRL IRL.18_1 26 0
IRL IRL.5_1 26 0
LBR LBR.

THA THA.5_1 77 21
THA THA.68_1 77 15
THA THA.62_1 77 18
THA THA.77_1 77 9
THA THA.72_1 77 23
THA THA.8_1 77 16
THA THA.24_1 77 13
THA THA.54_1 77 20
THA THA.1_1 77 7
THA THA.17_1 77 26
THA THA.15_1 77 11
THA THA.44_1 77 11
THA THA.14_1 77 18
THA THA.25_1 77 7
THA THA.45_1 77 9
THA THA.65_1 77 9
THA THA.21_1 77 14
THA THA.34_1 77 6
THA THA.73_1 77 18
THA THA.28_1 77 12
THA THA.56_1 77 18
THA THA.75_1 77 9
THA THA.10_1 77 24
THA THA.19_1 77 13
THA THA.20_1 77 8
THA THA.23_1 77 7
THA THA.35_1 77 9
THA THA.47_1 77 7
THA THA.4_1 77 8
THA THA.32_1 77 15
THA THA.41_1 77 8
THA THA.11_1 77 18
THA THA.76_1 77 7
THA THA.33_1 77 13
THA THA.64_1 77 16
THA THA.61_1 77 7
THA THA.38_1 77 6
THA THA.70_1 77 10
THA THA.40_1 77 12
THA THA.18_1 77 8
THA THA.48_1 77 2
THA THA.31_1 77 23
THA THA.39_1 77 7
THA THA.67_1 77 19
THA THA.51_1 77 5
THA THA.13_1 77 8
THA THA.50_1 77 8
THA THA.71_1 77 6
THA THA.9_1 77 10
THA THA.43_1 77 8
THA THA.12_1 77 10
THA THA.53_1 77 8
THA THA.52_1 77 9
THA THA.6_1 77 10
THA TH

TZA TZA.27_1 30 10
TZA TZA.28_1 30 1
TZA TZA.11_1 30 7
TZA TZA.19_1 30 2
TZA TZA.18_1 30 2
TZA TZA.23_1 30 5
TUN TUN.22_1 24 5
TUN TUN.10_1 24 5
TUN TUN.5_1 24 8
TUN TUN.14_1 24 7
TUN TUN.6_1 24 11
TUN TUN.17_1 24 13
TUN TUN.18_1 24 12
TUN TUN.9_1 24 12
TUN TUN.8_1 24 11
TUN TUN.12_1 24 10
TUN TUN.20_1 24 11
TUN TUN.15_1 24 5
TUN TUN.19_1 24 11
TUN TUN.11_1 24 10
TUN TUN.24_1 24 6
TUN TUN.7_1 24 8
TUN TUN.2_1 24 9
TUN TUN.16_1 24 13
TUN TUN.3_1 24 3
TUN TUN.13_1 24 5
TUN TUN.4_1 24 10
TUN TUN.23_1 24 1
TUN TUN.1_1 24 4
TUN TUN.21_1 24 7
CAF CAF.16_1 17 2
CAF CAF.7_1 17 3
CAF CAF.8_1 17 3
CAF CAF.12_1 17 4
CAF CAF.9_1 17 4
CAF CAF.3_1 17 4
CAF CAF.2_1 17 1
CAF CAF.13_1 17 5
CAF CAF.4_1 17 3
CAF CAF.11_1 17 3
CAF CAF.6_1 17 2
CAF CAF.14_1 17 3
CAF CAF.5_1 17 3
CAF CAF.15_1 17 5
CAF CAF.10_1 17 2
CAF CAF.1_1 17 2
CAF CAF.17_1 17 2
COM COM.2_1 3 0
COM COM.3_1 3 0
COM COM.1_1 3 0
CMR CMR.3_1 10 4
CMR CMR.10_1 10 4
CMR CMR.2_1 10 10
CMR CMR.5_1 10 4
CMR CMR.9_1 10 6
CMR CMR.8_1 10 8
CMR CMR.

TUR TUR.19_1 81 9
TUR TUR.23_1 81 12
TUR TUR.6_1 81 7
TUR TUR.66_1 81 12
TUR TUR.63_1 81 16
TUR TUR.79_1 81 4
TUR TUR.75_1 81 17
TUR TUR.28_1 81 9
TUR TUR.73_1 81 9
TUR TUR.52_1 81 7
TUR TUR.65_1 81 9
TUR TUR.10_1 81 8
TUR TUR.27_1 81 7
TUR TUR.9_1 81 6
TUR TUR.40_1 81 18
TUR TUR.43_1 81 6
TUR TUR.46_1 81 20
TUR TUR.67_1 81 15
TUR TUR.81_1 81 6
TUR TUR.50_1 81 8
TUR TUR.13_1 81 4
TUR TUR.70_1 81 9
MYS MYS.14_1 14 31
MYS MYS.1_1 14 10
MYS MYS.6_1 14 3
MYS MYS.7_1 14 7
MYS MYS.8_1 14 11
MYS MYS.15_1 14 9
MYS MYS.9_1 14 10
MYS MYS.16_1 14 7
MYS MYS.13_1 14 24
MYS MYS.3_1 14 10
MYS MYS.11_1 14 4
MYS MYS.2_1 14 11
MYS MYS.5_1 14 1
MYS MYS.10_1 14 1
XNC XNC.3_1 5 0
XNC XNC.5_1 5 0
XNC XNC.1_1 5 0
XNC XNC.2_1 5 0
XNC XNC.4_1 5 0
DEU DEU.2_1 16 87
DEU DEU.1_1 16 43
DEU DEU.11_1 16 32
DEU DEU.12_1 16 6
DEU DEU.7_1 16 26
DEU DEU.14_1 16 13
DEU DEU.16_1 16 22
DEU DEU.10_1 16 51
DEU DEU.13_1 16 14
DEU DEU.9_1 16 46
DEU DEU.4_1 16 18
DEU DEU.3_1 16 1
DEU DEU.5_1 16 2
DEU DEU.8_1 16 8
DEU DEU.6_1 16

SRB SRB.17_1 25 3
SRB SRB.18_1 25 6
MTQ MTQ.3_1 4 2
MTQ MTQ.2_1 4 1
MTQ MTQ.1_1 4 1
MTQ MTQ.4_1 4 1
AND AND.5_1 4 0
AND AND.3_1 4 0
AND AND.6_1 4 0
AND AND.2_1 4 0
SDN SDN.8_1 18 5
SDN SDN.9_1 18 5
SDN SDN.7_1 18 7
SDN SDN.1_1 18 7
SDN SDN.2_1 18 5
SDN SDN.6_1 18 5
SDN SDN.12_1 18 6
SDN SDN.10_1 18 4
SDN SDN.11_1 18 4
SDN SDN.14_1 18 5
SDN SDN.17_1 18 5
SDN SDN.5_1 18 2
SDN SDN.3_1 18 5
SDN SDN.15_1 18 5
SDN SDN.4_1 18 2
SDN SDN.13_1 18 3
SDN SDN.18_1 18 4
SDN SDN.16_1 18 1
MOZ MOZ.8_1 11 15
MOZ MOZ.1_1 11 15
MOZ MOZ.6_1 11 7
MOZ MOZ.5_1 11 1
MOZ MOZ.2_1 11 11
MOZ MOZ.3_1 11 12
MOZ MOZ.4_1 11 9
MOZ MOZ.10_1 11 12
MOZ MOZ.11_1 11 16
MOZ MOZ.7_1 11 18
MOZ MOZ.9_1 11 12
MMR MMR.9_1 15 2
MMR MMR.6_1 15 3
MMR MMR.1_1 15 5
MMR MMR.15_1 15 3
MMR MMR.2_1 15 4
MMR MMR.11_1 15 5
MMR MMR.5_1 15 2
MMR MMR.7_1 15 4
MMR MMR.8_1 15 6
MMR MMR.13_1 15 11
MMR MMR.10_1 15 1
MMR MMR.12_1 15 8
MMR MMR.4_1 15 3
MMR MMR.14_1 15 3
MMR MMR.3_1 15 2
VUT VUT.4_1 6 6
VUT VUT.1_1 6 8
VUT VUT.2_1 6 4
VUT VUT.3_1 6 

MEX MEX.25_1 32 17
MEX MEX.10_1 32 38
MEX MEX.32_1 32 55
MEX MEX.24_1 32 53
MEX MEX.19_1 32 47
MEX MEX.28_1 32 42
MEX MEX.7_1 32 38
MEX MEX.6_1 32 66
MEX MEX.26_1 32 69
MEX MEX.3_1 32 5
MEX MEX.11_1 32 41
MEX MEX.22_1 32 18
MEX MEX.18_1 32 19
MEX MEX.1_1 32 9
MKD MKD.42_1 73 0
MKD MKD.61_1 73 0
MKD MKD.4_1 73 0
MKD MKD.50_1 73 0
MKD MKD.52_1 73 0
MKD MKD.68_1 73 0
MKD MKD.48_1 73 0
MKD MKD.57_1 73 0
MKD MKD.17_1 73 0
MKD MKD.19_1 73 0
MKD MKD.32_1 73 0
MKD MKD.24_1 73 0
MKD MKD.5_1 73 0
MKD MKD.65_1 73 0
MKD MKD.40_1 73 0
MKD MKD.75_1 73 0
MKD MKD.22_1 73 0
MKD MKD.21_1 73 0
MKD MKD.20_1 73 0
MKD MKD.69_1 73 0
MKD MKD.51_1 73 0
MKD MKD.13_1 73 0
MKD MKD.47_1 73 0
MKD MKD.80_1 73 0
MKD MKD.46_1 73 0
MKD MKD.11_1 73 0
MKD MKD.62_1 73 0
MKD MKD.49_1 73 0
MKD MKD.36_1 73 0
MKD MKD.76_1 73 0
MKD MKD.7_1 73 0
MKD MKD.82_1 73 0
MKD MKD.53_1 73 0
MKD MKD.27_1 73 0
MKD MKD.67_1 73 0
MKD MKD.59_1 73 0
MKD MKD.3_1 73 0
MKD MKD.26_1 73 0
MKD MKD.77_1 73 0
MKD MKD.64_1 73 0
MKD MKD.70_1 73 0
MKD MK

In [79]:
# dump dict to pickle file
with open('input/dicts/adm_2_to_grid.pkl', 'wb') as handle:
    pickle.dump(adm_2_to_grid, handle)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>